In [ ]:
import os
import pickle
import numpy as np
import torch
import pandas as pd
import time
import lightning_fabric.utilities.data as lf_data
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, LSTM, NHITS
from neuralforecast.losses.pytorch import MAE

# --- 0. 설정 및 경로 정의 ---
KS_MODEL_PATH = './output/kshape_model_k6.pkl'
TEST_DATA_PATH = './dataset/preprocessed_test_dataset.npy'
MODELS_DIR = './models'

# 클러스터 번호에 따른 전문가 모델 경로 매핑
EXPERT_MODEL_PATHS = {
    0: os.path.join(MODELS_DIR, 'N-BEATS/cluster_0_nbeats/NBEATS_0.ckpt'),
    1: os.path.join(MODELS_DIR, 'LSTM/cluster_1_lstm/LSTM_0.ckpt'),
    2: os.path.join(MODELS_DIR, 'N-HiTS/cluster_2_nhits/NHITS_0.ckpt'),
    3: os.path.join(MODELS_DIR, 'N-BEATS/cluster_3_nbeats/NBEATS_0.ckpt'),
    4: os.path.join(MODELS_DIR, 'LSTM/cluster_4_lstm/LSTM_0.ckpt'),
    5: os.path.join(MODELS_DIR, 'N-HiTS/cluster_5_nhits/NHITS_0.ckpt'),
}

# 예측에 필요한 상수
INPUT_SIZE = 23 * 60  # 입력 기간 (23시간)
HORIZON = 1 * 60      # 예측 기간 (1시간)
MAX_EPOCHS = 100
INTERVAL = 1
HIDDEN_SIZE = 16

# 1. 모델 인스턴스 미리 생성
lstm_model = LSTM(
    h=HORIZON,
    input_size=INPUT_SIZE,
    encoder_hidden_size=HIDDEN_SIZE,
    decoder_hidden_size=HIDDEN_SIZE,
    max_steps=MAX_EPOCHS
)
nbeats_model = NBEATS(input_size=INPUT_SIZE, h=HORIZON, max_steps=MAX_EPOCHS)
nhits_model = NHITS(input_size=INPUT_SIZE, h=HORIZON, max_steps=MAX_EPOCHS)

# 2. 모델 딕셔너리로 관리
expert_models = {
    'LSTM': lstm_model,
    'NBEATS': nbeats_model,
    'NHITS': nhits_model
}

# 3. cluster_label에 따라 모델 선택
cluster_to_model_type = {
    0: 'NBEATS',
    1: 'LSTM',
    2: 'NHITS',
    3: 'NBEATS',
    4: 'LSTM',
    5: 'NHITS'
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"--- Using accelerator: {device} ---")

Seed set to 1
Seed set to 1
Seed set to 1


--- Using accelerator: cpu ---


In [ ]:
# --- 1. K-Shape 모델로 클러스터 예측 (Gating Network) ---
def k_shape_predict(dataset):
    """K-Shape 모델을 로드하여 데이터셋의 클러스터를 예측합니다."""
    with open(KS_MODEL_PATH, 'rb') as f:
        model = pickle.load(f)
    cluster_labels = model.predict(dataset)
    print(f"클러스터 예측 완료. 클러스터 분포: {pd.Series(cluster_labels).value_counts().to_dict()}")
    return cluster_labels

In [3]:
# --- 2. 전문가 모델 로드 및 예측 ---
# 전문가 모델을 효율적으로 관리하기 위해 캐싱(메모리에 저장)합니다.

def preprocess_for_nf(sample_input):
    input_size = INPUT_SIZE
    base_date='2025-01-01'
    arr = np.asarray(sample_input).squeeze()
    print(f"arr: {arr}")
    data_range = pd.date_range(base_date, periods=input_size, freq='min')
    df = pd.DataFrame({
        'unique_id': 'sample',
        'ds': data_range,
        'y': arr
    })
    return df

loaded_experts = {}

def predict_with_expert(cluster_label, input_data):
    "선택된 전문가 모델로 예측을 수행합니다."
    # 1. 캐시에서 모델 가져오기
    if cluster_label in loaded_experts:
        model = loaded_experts[cluster_label]
        print(f"loaded_experts[cluster_label]: {loaded_experts[cluster_label]}")
    else:
        model_path = EXPERT_MODEL_PATHS.get(cluster_label)
        torch.serialization.add_safe_globals([lf_data.AttributeDict, MAE])
        checkpoint = torch.load(model_path, weights_only=True)
        hyper_params = checkpoint['hyper_parameters']
        state_dict = checkpoint['state_dict']

        if cluster_label in [0, 3]: # N-BEATS
            model = NBEATS(**hyper_params)
        elif cluster_label in [1, 4]: # LSTM
            model = LSTM(**hyper_params)
        elif cluster_label in [2, 5]: # N-HiTS
            model = NHITS(**hyper_params)
        else:
            raise ValueError(f"Unknown cluster label: {cluster_label}")
        
        model.load_state_dict(state_dict)
        model.eval()
        loaded_experts[cluster_label] = model
        print(f"{cluster_label} 번 전문가 모델 {model} 로드 및 캐시 완료")
    
    # 2. 예측
    df_test = preprocess_for_nf(input_data)
    nf = NeuralForecast(models=[model], freq='min')
    ## 초기값 설정
    nf.id_col = 'unique_id'
    nf.time_col = 'ds'
    nf.target_col = 'y'
    nf.scalers_ = None
    ## 학습은 생략
    nf._fitted = True

    prediction = nf.predict(df=df_test)
    return prediction

In [ ]:
# --- 3. 메인 MoE 추론 및 평가 파이프라인 ---
X_test_full = np.load(TEST_DATA_PATH)  # 전체 테스트 데이터 로드

# 빠른 테스트를 위해 일부만 사용
X_test_full = X_test_full

# 입력 데이터(처음 23시간)와 실제 정답(마지막 1시간) 분리
X_test_input = X_test_full[:, :INPUT_SIZE, :]
y_test_true = X_test_full[:, INPUT_SIZE:, :]

# --- MoE 추론 시작 ---
start_time = time.time()

# 1. Gating: 모든 테스트 데이터에 대해 클러스터를 미리 예측
all_cluster_labels = k_shape_predict(X_test_full) # all_cluaster_labels을 구할 때는 전체 데이터 사용
print(f"   -> 예측된 클러스터 분포: {np.unique(all_cluster_labels, return_counts=True)}")

# 2. Routing & Prediction: 각 데이터 샘플에 대해 전문가 모델로 예측 수행
all_predictions = []
for i in range(len(X_test_input)):
    sample_input = X_test_input[i]
    cluster_label = all_cluster_labels[i]

    # 해당 전문가로 예측
    prediction = predict_with_expert(cluster_label, sample_input)
    all_predictions.append(prediction)
    if (i+1) % 10 == 0 or i == len(X_test_input)-1:
        print(f"  [{i+1}/{len(X_test_input)}] 샘플 예측 완료")

# Numpy 배열로 변환
y_pred_moe = np.array(all_predictions)

end_time = time.time()
inference_time = end_time - start_time
print(f"MoE 모델 예측 결과: {y_pred_moe.shape}")

In [ ]:
# --- 4. 평가 지표 계산 ---
y_true_final = y_test_true.squeeze(-1)
y_pred_final = np.array([
    [row[-1] for row in sample] for sample in y_pred_moe
])
y_pred = np.array([[row[-1] for row in y_pred_moe]]) # y_pred: (샘플 수, HORIZON, 3) => 예측값만 추출
print(f"y_true shape: {y_true_final.shape}, y_pred shape: {y_pred_final.shape}")

# MAE 계산
mae_per_sample = np.mean(np.abs(y_true_final - y_pred_final), axis=1)
mae = np.mean(mae_per_sample)
print(f"MoE 모델 MAE: {mae:.4f}")

# RMAE
rmse_per_sample = np.sqrt(np.mean((y_true_final - y_pred_final) ** 2, axis=1))
rmse = np.mean(rmse_per_sample)
print(f"MoE 모델 RMSE: {rmse:.4f}")

# MAPE
mape_list = []
for yt, yp in zip(y_true_final, y_pred_final):
    mask = np.abs(yt) > 1e-1
    if np.any(mask):
        mape = np.mean(np.abs((yt[mask] - yp[mask]) / (yt[mask] + 1e-8))) * 100
        mape_list.append(mape)
    else:
        mape_list.append(np.nan)  # 실제값이 모두 0이면 NaN

mape = np.nanmean(mape_list)
print(f"MoE 모델 MAPE: {mape:.4f}%")

print(f"MoE 모델 추론 시간: {inference_time:.2f}초")
print(f"샘플 당 평균 추론 시간: {(inference_time / len(X_test_input))*1000:.4f} ms")

y_true shape: (3, 60), y_pred shape: (3, 60)
MoE 모델 MAE: 0.3426
MoE 모델 RMSE: 0.4355
MoE 모델 MAPE: 75.6388%
MoE 모델 추론 시간: 1.33초
샘플 당 평균 추론 시간: 443.0676 ms
